In [2]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

#### numpy[:, -1] 과 numpy[:, [-1]] 의 차이

In [3]:
test_numpy = [[1,2,3], [4,5,6], [7,8,9]]
test_numpy = np.array(test_numpy)
test_numpy

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [4]:
test_numpy[:, 0:-1]

array([[1, 2],
       [4, 5],
       [7, 8]])

In [5]:
test_numpy[:, -1]

array([3, 6, 9])

In [6]:
test_numpy[:, [-1]]

array([[3],
       [6],
       [9]])

#### 다시 torch 부분

In [7]:
class DiabetesDataset(Dataset):
    
    def __init__(self):
        xy = np.loadtxt('./data/diabetes.csv.gz',
                       delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [8]:
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                         batch_size=32,
                         shuffle=True,
                         num_workers=2)

In [9]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8,6)
        self.l2 = torch.nn.Linear(6,4)
        self.l3 = torch.nn.Linear(4,1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
    
model = Model()

In [10]:
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [11]:
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        inputs, labels = Variable(inputs), Variable(labels)
        
        y_pred = model(inputs)
        
        loss = criterion(y_pred, labels)
        print(epoch, i ,loss.data[0])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 